# Copula Practise

In [51]:
import pypistats
import pandas as pd
import json  

libraries = ["copulas", "copulae", "pymc3", "openturns", "pyvinecopulib", "scikit-copula"]
downloads = []

for lib in libraries:
    try:
        # Get the JSON as a string
        data_str = pypistats.recent(lib, format="json")
        
        # Parse the JSON string into a Python dictionary
        data = json.loads(data_str)
        
        # Now you can access the data structure properly
        monthly_downloads = data["data"]["last_month"]
        downloads.append({"library": lib, "monthly_downloads": monthly_downloads})
    except Exception as e:
        print(f"Error for {lib}: {str(e)}")
        downloads.append({"library": lib, "monthly_downloads": "Error retrieving data"})
        
pd.DataFrame(downloads)

Error for scikit-copula: Client error '404 Not Found' for url 'https://pypistats.org/api/packages/scikit-copula/recent'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404


,library,monthly_downloads
0,copulas,196183
1,copulae,4632
2,pymc3,292670
3,openturns,20935
4,pyvinecopulib,21825
5,scikit-copula,Error retrieving data


## Univariate Copulae

In [1]:
from copulas.datasets import sample_univariate_beta

data = sample_univariate_beta()
data.head()

from copulas.visualization import dist_1d

dist_1d(data)


In [2]:

from copulas.univariate import BetaUnivariate

beta = BetaUnivariate()
beta.fit(data)
beta._params

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt



{'loc': np.float64(4.052009283524141),
 'scale': np.float64(0.947975527152111),
 'a': np.float64(2.5523231909067743),
 'b': np.float64(0.9041091899047304)}

In [3]:
sampled = beta.sample(1000)
from copulas.visualization import compare_1d

compare_1d(data, sampled)

In [4]:
probability_density = beta.pdf(sampled)
probability_density[0:5]


array([0.997117  , 2.03209631, 1.07396457, 0.99319086, 2.67621692])

In [5]:
import pandas as pd

pd.options.plotting.backend = "plotly"
pd.DataFrame({
    'data': sampled,
    'probability_density': probability_density
}).sort_values('data').set_index('data').plot()



In [6]:
parameters = beta.to_dict()
parameters


{'loc': np.float64(4.052009283524141),
 'scale': np.float64(0.947975527152111),
 'a': np.float64(2.5523231909067743),
 'b': np.float64(0.9041091899047304),
 'type': 'copulas.univariate.beta.BetaUnivariate'}

In [7]:
from copulas.univariate import Univariate

new_beta = Univariate.from_dict(parameters)

new_sampled = new_beta.sample(1000)

compare_1d(data, new_sampled)

In [8]:
from copulas.univariate import Univariate

univariate = Univariate()
univariate.fit(data)


/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt



In [9]:
parameters = univariate.to_dict()
parameters

{'loc': np.float64(4.052009283524141),
 'scale': np.float64(0.947975527152111),
 'a': np.float64(2.5523231909067743),
 'b': np.float64(0.9041091899047304),
 'type': 'copulas.univariate.beta.BetaUnivariate'}

In [11]:
from copulas.datasets import sample_univariates

data = sample_univariates()
data.head()


,bernoulli,bimodal,uniform,normal,degenerate,exponential,beta
0,0.0,11.399355,0.498160,1.496714,0.37454,3.469268,4.796025
1,0.0,10.924634,2.802857,0.861736,0.37454,6.010121,4.935189
2,0.0,10.059630,1.927976,1.647689,0.37454,4.316746,4.637677
3,0.0,9.353063,1.394634,2.523030,0.37454,3.912943,4.945320
4,1.0,-0.234153,-0.375925,0.765847,0.37454,3.169625,4.726815


In [13]:
synth_data = pd.DataFrame()
distributions = []

for column in data.columns:
    real_data = data[column]
    univariate = Univariate()
    univariate.fit(real_data)
    synth_data[column] = univariate.sample(len(real_data))
    distributions.append(univariate.to_dict()['type'])
synth_data.head()



/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:800: RuntimeWarning:

The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:800: RuntimeWarning:

The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:800: RuntimeWarning:

The iteration is not making good progress, as 

,bernoulli,bimodal,uniform,normal,degenerate,exponential,beta
0,-0.304536,12.232326,0.525182,0.176463,0.37454,4.285615,4.855229
1,-0.012735,0.841163,1.187476,1.411188,0.37454,3.895427,4.957942
2,0.077222,9.288727,1.550949,2.021484,0.37454,6.122943,4.737354
3,0.212315,0.347036,-0.445246,-0.496678,0.37454,3.073039,4.711186
4,1.024265,13.497303,-0.318220,0.170632,0.37454,3.283878,4.754044


In [14]:
distributions

['copulas.univariate.gaussian_kde.GaussianKDE',
 'copulas.univariate.gaussian_kde.GaussianKDE',
 'copulas.univariate.beta.BetaUnivariate',
 'copulas.univariate.gaussian_kde.GaussianKDE',
 'copulas.univariate.beta.BetaUnivariate',
 'copulas.univariate.beta.BetaUnivariate',
 'copulas.univariate.beta.BetaUnivariate']

In [15]:
for column in data.columns:
    fig = compare_1d(data[column], synth_data[column])
    fig.show()



In [16]:
univariate = Univariate()
univariate.fit(data['beta'])
new_model = Univariate.from_dict(parameters)



/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt



In [17]:
new_model.__class__

copulas.univariate.beta.BetaUnivariate

In [18]:
from copulas.univariate import GaussianUnivariate, GammaUnivariate

candidates = [BetaUnivariate, GaussianUnivariate, GammaUnivariate]

univariate = Univariate(candidates=candidates)
univariate.fit(data['bimodal'])

univariate.to_dict()

{'loc': np.float64(6.830066099361434),
 'scale': np.float64(4.811082463652564),
 'type': 'copulas.univariate.gaussian.GaussianUnivariate'}

In [19]:
from copulas.univariate import GaussianKDE, TruncatedGaussian

univariates = [
    BetaUnivariate,
    GammaUnivariate,
    GaussianKDE,
    GaussianUnivariate,
    TruncatedGaussian
]

pd.DataFrame([
    [uni.__name__, uni.PARAMETRIC, uni.BOUNDED]
    for uni in univariates
], columns=['Distribution', 'Parametric', 'Bounded'])

,Distribution,Parametric,Bounded
0,BetaUnivariate,ParametricType.PARAMETRIC,BoundedType.BOUNDED
1,GammaUnivariate,ParametricType.PARAMETRIC,BoundedType.SEMI_BOUNDED
2,GaussianKDE,ParametricType.NON_PARAMETRIC,BoundedType.UNBOUNDED
3,GaussianUnivariate,ParametricType.PARAMETRIC,BoundedType.UNBOUNDED
4,TruncatedGaussian,ParametricType.PARAMETRIC,BoundedType.BOUNDED


In [ ]:
from copulas.univariate import ParametricType, BoundedType

univariate = Univariate(
    parametric=ParametricType.PARAMETRIC,
    bounded=BoundedType.BOUNDED
)
univariate.fit(data['bimodal'])

univariate.to_dict()



{'a': np.float64(-1.5932081561011588),
 'b': np.float64(0.0),
 'loc': np.float64(13.19310768705415),
 'scale': np.float64(10.315271788810145),
 'type': 'copulas.univariate.truncated_gaussian.TruncatedGaussian'}

## Multivariate Distributions

In [21]:
from copulas.datasets import sample_trivariate_xyz

data = sample_trivariate_xyz()
data.head()




,x,y,z
0,9.004177e-05,2.883992e-06,0.638689
1,8.819273e-01,2.911979e-07,1.058121
2,5.003865e-01,4.886504e-04,0.372506
3,1.838544e-12,5.392802e-02,0.687370
4,1.627915e-01,1.634269e-08,-0.881068


In [22]:
from copulas.multivariate import GaussianMultivariate

dist = GaussianMultivariate()
dist.fit(data)



/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:6720: RuntimeWarning:

divide by zero encountered in power



In [24]:
sampled = dist.sample(1000)
sampled.head()

,x,y,z
0,0.042535,5.186650e-07,0.026497
1,0.415040,3.513779e-01,0.860691
2,0.394505,1.092242e-02,0.696746
3,0.927826,4.924341e-05,1.827555
4,1.000000,9.321223e-06,-0.191437


In [27]:
from copulas.visualization import compare_3d

compare_3d(data, sampled)



In [28]:
from copulas.univariate import GaussianUnivariate

dist = GaussianMultivariate(distribution=GaussianUnivariate)
dist.fit(data)
sampled = dist.sample(1000)

compare_3d(data, sampled)

In [29]:
from copulas.univariate import BetaUnivariate, GaussianKDE, GaussianUnivariate

dist = GaussianMultivariate(distribution={
    "x": BetaUnivariate,
    "y": GaussianKDE,
    "z": GaussianUnivariate,
})
dist.fit(data)
sampled = dist.sample(1000)

compare_3d(data, sampled)

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt



In [30]:
from copulas.univariate import ParametricType, Univariate

# Select the best PARAMETRIC univariate
univariate = Univariate(parametric=ParametricType.PARAMETRIC)

dist = GaussianMultivariate(distribution=univariate)
dist.fit(data)
sampled = dist.sample(1000)

compare_3d(data, sampled)



/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:795: RuntimeWarning:

invalid value encountered in sqrt

/home/cjrisi/Projects/diabetes/simglucose/.newvp/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:6720: RuntimeWarning:

divide by zero encountered in power



## Synthetic Data for Machine Learning


In [31]:
import warnings

warnings.filterwarnings('ignore')

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [32]:
import numpy as np

from copulas.multivariate import GaussianMultivariate

def create_synthetic(X, y):
    """
    This function combines X and y into a single dataset D, models it
    using a Gaussian copula, and generates a synthetic dataset S. It
    returns the new, synthetic versions of X and y.
    """
    dataset = np.concatenate([X, np.expand_dims(y, 1)], axis=1)

    model = GaussianMultivariate()
    model.fit(dataset)

    synthetic = model.sample(len(dataset))

    X = synthetic.values[:, :-1]
    y = synthetic.values[:, -1]

    return X, y

X_synthetic, y_synthetic = create_synthetic(X_train, y_train)

In [34]:
from sklearn.linear_model import ElasticNet

model = ElasticNet()
model.fit(X_synthetic, y_synthetic)

ElasticNet()

In [35]:
print(model.score(X_test, y_test))



-0.06384694043474037


In [36]:
model = ElasticNet()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

-0.025490778188169694


## Contructing a Copula Vine Tree

In [53]:
from copulas.datasets import sample_trivariate_xyz

data = sample_trivariate_xyz()

data.head()

,x,y,z
0,9.004177e-05,2.883992e-06,0.638689
1,8.819273e-01,2.911979e-07,1.058121
2,5.003865e-01,4.886504e-04,0.372506
3,1.838544e-12,5.392802e-02,0.687370
4,1.627915e-01,1.634269e-08,-0.881068


In [ ]:
from copulas.visualization import scatter_3d

scatter_3d(data)



In [56]:
sampled = dist.sample(1000)
sampled.head()

,x,y,z
0,1.118944e-07,9.219073e-01,3.430891
1,1.717075e-05,2.546437e-02,3.040637
2,1.873417e-02,1.040828e-07,1.593149
3,9.465388e-01,1.150313e-08,-0.091232
4,3.475722e-05,6.369121e-01,1.801474


In [57]:
from copulas.visualization import compare_3d

compare_3d(data, sampled)



In [58]:
from copulas.univariate import GaussianUnivariate

dist = GaussianMultivariate(distribution=GaussianUnivariate)
dist.fit(data)
sampled = dist.sample(1000)

compare_3d(data, sampled)



In [59]:
from copulas.univariate import BetaUnivariate, GaussianKDE, GaussianUnivariate

dist = GaussianMultivariate(distribution={
    "x": BetaUnivariate,
    "y": GaussianKDE,
    "z": GaussianUnivariate,
})
dist.fit(data)
sampled = dist.sample(1000)

compare_3d(data, sampled)

In [60]:
from copulas.univariate import ParametricType, Univariate

# Select the best PARAMETRIC univariate
univariate = Univariate(parametric=ParametricType.PARAMETRIC)

dist = GaussianMultivariate(distribution=univariate)
dist.fit(data)
sampled = dist.sample(1000)

compare_3d(data, sampled)

In [61]:
parameters = dist.to_dict()
parameters.keys()


dict_keys(['correlation', 'univariates', 'columns', 'type'])

In [62]:
parameters['correlation']



[[1.0, -0.02198490877903673, -0.03904683794009118],
 [-0.02198490877903673, 1.0, 0.7099236519163041],
 [-0.03904683794009118, 0.7099236519163041, 1.0]]

In [63]:
parameters['univariates']



[{'loc': np.float64(1.3765984634140437e-23),
  'scale': np.float64(1.0000000000000002),
  'a': np.float64(0.09657613485947697),
  'b': np.float64(0.10226371906555821),
  'type': 'copulas.univariate.beta.BetaUnivariate'},
 {'loc': np.float64(2.5976191218752e-49),
  'scale': np.float64(0.9999824369833828),
  'a': np.float64(0.11005630929075551),
  'b': np.float64(0.5585743539945622),
  'type': 'copulas.univariate.beta.BetaUnivariate'},
 {'df': np.float64(1.4006628830710697),
  'loc': np.float64(0.4691588432897164),
  'scale': np.float64(1.247945331065269),
  'type': 'copulas.univariate.student_t.StudentTUnivariate'}]

In [64]:
from copulas.multivariate import Multivariate

new_dist = Multivariate.from_dict(parameters)

In [65]:
new_dist


GaussianMultivariate()

In [66]:
from copulas.multivariate import VineCopula

center = VineCopula('center')
regular = VineCopula('regular')
direct = VineCopula('direct')

center.fit(data)
regular.fit(data)
direct.fit(data)

center_samples = center.sample(1000)
regular_samples = regular.sample(1000)
direct_samples = direct.sample(1000)

In [67]:
scatter_3d(data, title='Real Data')



In [68]:
scatter_3d(regular_samples, title='R-Vine')

In [73]:
vine_example = regular.to_dict()

In [74]:
vine_example.keys()

dict_keys(['type', 'vine_type', 'fitted', 'n_sample', 'n_var', 'depth', 'truncated', 'trees', 'tau_mat', 'u_matrix', 'unis', 'columns'])

In [102]:
vine_example['unis'][0]['type']

'copulas.univariate.gaussian_kde.GaussianKDE'